In [1]:
import pyspark
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Classification").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler, OneHotEncoder, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [3]:
schema = StructType([
    StructField("age", FloatType(), True),
    StructField("sex", FloatType(), True),
    StructField("chest_pain_type", FloatType(), True),
    StructField("resting_bps", FloatType(), True),
    StructField("chol", FloatType(), True),
    StructField("fast_blood_sugar", FloatType(), True),
    StructField("rest_ecg_type", FloatType(), True),
    StructField("max_hr", FloatType(), True),
    StructField("exercise_angina", FloatType(), True),
    StructField("oldpeak", FloatType(), True),
    StructField("slope_type", FloatType(), True),
    StructField("colored_arteries", FloatType(), True),
    StructField("thal_type", FloatType(), True),
    StructField("heart_disease", IntegerType(), True),
])

df = spark.read.csv("data/processed.cleveland.data", schema = schema, header = False, nullValue="?")
df.limit(6).toPandas()

,age,sex,chest_pain_type,resting_bps,chol,fast_blood_sugar,rest_ecg_type,max_hr,exercise_angina,oldpeak,slope_type,colored_arteries,thal_type,heart_disease
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
5,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0


In [4]:
print(df.describe().toPandas())

  summary                age                  sex     chest_pain_type  \
0   count                303                  303                 303   
1    mean  54.43894389438944   0.6798679867986799  3.1584158415841586   
2  stddev   9.03866244244675  0.46729882777012977  0.9601256119600138   
3     min               29.0                  0.0                 1.0   
4     max               77.0                  1.0                 4.0   

          resting_bps                chol    fast_blood_sugar  \
0                 303                 303                 303   
1  131.68976897689768  246.69306930693068  0.1485148514851485   
2   17.59974772958769  51.776917542637065  0.3561978749279763   
3                94.0               126.0                 0.0   
4               200.0               564.0                 1.0   

        rest_ecg_type              max_hr      exercise_angina  \
0                 303                 303                  303   
1  0.9900990099009901   149.6072607260

In [5]:
df.groupBy("heart_disease").count().show()

+-------------+-----+
|heart_disease|count|
+-------------+-----+
|            1|   55|
|            3|   35|
|            4|   13|
|            2|   36|
|            0|  164|
+-------------+-----+



In [6]:
#check how much missing data there is
dataAgg = df.agg(*[count(when(isnull(c), c)).alias(c) for c in df.columns])
print(dataAgg.limit(8).toPandas())

dfClean = df.na.drop()
features = df.columns[:-1]
label = "heart_disease"

   age  sex  chest_pain_type  resting_bps  chol  fast_blood_sugar  \
0    0    0                0            0     0                 0   

   rest_ecg_type  max_hr  exercise_angina  oldpeak  slope_type  \
0              0       0                0        0           0   

   colored_arteries  thal_type  heart_disease  
0                 4          2              0  


In [7]:
# change from multiclass to binary prediction
dfBinary = dfClean.withColumn("label", when(df.heart_disease == 0, 0).otherwise(1))
print(dfBinary.limit(10).show())

categoricalCols = [col for col in df.columns if "type" in col]
print(categoricalCols)

continuousCols = [f for f in features if "type" not in f]
continuousCols.remove("sex")

+----+---+---------------+-----------+-----+----------------+-------------+------+---------------+-------+----------+----------------+---------+-------------+-----+
| age|sex|chest_pain_type|resting_bps| chol|fast_blood_sugar|rest_ecg_type|max_hr|exercise_angina|oldpeak|slope_type|colored_arteries|thal_type|heart_disease|label|
+----+---+---------------+-----------+-----+----------------+-------------+------+---------------+-------+----------+----------------+---------+-------------+-----+
|63.0|1.0|            1.0|      145.0|233.0|             1.0|          2.0| 150.0|            0.0|    2.3|       3.0|             0.0|      6.0|            0|    0|
|67.0|1.0|            4.0|      160.0|286.0|             0.0|          2.0| 108.0|            1.0|    1.5|       2.0|             3.0|      3.0|            2|    1|
|67.0|1.0|            4.0|      120.0|229.0|             0.0|          2.0| 129.0|            1.0|    2.6|       2.0|             2.0|      7.0|            1|    1|
|37.0|1.0|

In [9]:
#create dense feature vector and scale
featuresList = continuousCols + categoricalCols

assembler = VectorAssembler(inputCols = featuresList, outputCol = "features")
indexer = VectorIndexer(inputCol="features", outputCol="indexed", maxCategories=5)
scaler = MinMaxScaler(inputCol="indexed", outputCol="features_scaled")
pipeline = Pipeline(stages=[assembler, indexer, scaler])
scalerModel = pipeline.fit(dfBinary)
scaledData = scalerModel.transform(dfBinary).select("features_scaled", "label")
scaledData = scaledData.withColumnRenamed("features_scaled", "features")
scaledData.limit(10).toPandas()

,features,label
0,"[0.7083333333333333, 0.4811320754716981, 0.244...",0
1,"[0.7916666666666666, 0.6226415094339622, 0.365...",1
2,"[0.7916666666666666, 0.24528301886792453, 0.23...",1
3,"[0.16666666666666666, 0.33962264150943394, 0.2...",0
4,"[0.25, 0.33962264150943394, 0.1780821917808219...",0
5,"(0.5625, 0.24528301886792453, 0.25114155251141...",0
6,"[0.6875, 0.43396226415094336, 0.32420091324200...",1
7,"[0.5833333333333333, 0.24528301886792453, 0.52...",0
8,"[0.7083333333333333, 0.33962264150943394, 0.29...",1
9,"[0.5, 0.43396226415094336, 0.17579908675799086...",1


In [10]:
trainPercent = 0.7
seed = 1837
train, test = scaledData.randomSplit([trainPercent, 1-trainPercent], seed = seed)
print(f"train len: {train.count()}, test len: {test.count()}")

train len: 201, test len: 96


## Add MLflow tracking

* Create an experiment
* Create a run / runs
* Save tags, parameters, metrics and artifacts for a run

In [11]:
import mlflow
import mlflow.spark

mlflow.set_experiment(experiment_name = "sd_ml")

#set up client
from mlflow.tracking import MlflowClient
client = MlflowClient()

In [12]:
# Create a run and attach it to the experiment you just created
experiments = client.list_experiments() 

experiment_name = "sd_ml"
def create_run(experiment_name):
    mlflow.set_experiment(experiment_name = experiment_name)
    for x in experiments:
        if experiment_name in x.name:
            experiment_index = experiments.index(x)
            run = client.create_run(experiments[experiment_index].experiment_id) # returns mlflow.entities.Run
            return run

In [ ]:
#just a demo of the functionality here
run = create_run(experiment_name)

# Add tag to a run
client.set_tag(run.info.run_id, "Algorithm", "Gradient Boosted Tree")
client.set_tag(run.info.run_id,"Random Seed", 999)
client.set_tag(run.info.run_id,"Train Perct", 999)

# Add params and metrics to a run
client.log_param(run.info.run_id, "Max Depth", 999)
client.log_param(run.info.run_id, "Max Bins", 999)
client.log_metric(run.info.run_id, "Accuracy", 999)

# Terminate the client
client.set_terminated(run.info.run_id)

## Train and Compare Models

In [ ]:
def selectParameters(Mtype):
    if Mtype == "LogisticRegression":
            paramGrid = (ParamGridBuilder() \
                         .addGrid(classifier.regParam, [0.1, 0.01]) \
                         .addGrid(classifier.maxIter, [10, 15,20])
                         .build())
    if Mtype == "RandomForestClassifier":
            paramGrid = (ParamGridBuilder() \
                         .addGrid(classifier.maxDepth, [2, 5, 10]) \
                         .addGrid(classifier.maxBins, [5, 10, 20]) \
                         .addGrid(classifier.numTrees, [5, 20])
                         .build())

    if Mtype == "GBTClassifier":
        paramGrid = (ParamGridBuilder() \
                     .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
                     .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                     .addGrid(classifier.maxIter, [10, 15]) \
                     .addGrid(classifier.stepSize, [0.1, 0.2, 0.3]) #learning rate
                     .build())
    return paramGrid

def logParameters(best_model):
    
    paramMap = best_model.extractParamMap()

    for key, val in paramMap.items():
        if "maxIter" in str(key):
            client.log_param(run.info.run_id, "Max Iter", val)
        elif "maxDepth" in str(key):
            client.log_param(run.info.run_id, "Max Depth", val)
        elif "maxBins" in str(key):
            client.log_param(run.info.run_id, "Max Bins", val)
        elif "regParam" in str(key):
            client.log_param(run.info.run_id, "Reg Params", val)
            

                
classifiers = [LogisticRegression(), GBTClassifier(), RandomForestClassifier()]

for classifier in classifiers:
    run = create_run(experiment_name)
    client.set_tag(run.info.run_id, "Algorithm", classifier)
    client.set_tag(run.info.run_id,"Random Seed",seed)
    client.set_tag(run.info.run_id,"Train Perct",trainPercent)
    
    Mtype = type(classifier).__name__
    print(Mtype)
    BC_evaluator = BinaryClassificationEvaluator() 
    paramGrid = selectParameters(Mtype)

    crossval = CrossValidator(
        estimator = classifier,
        estimatorParamMaps = paramGrid,
        evaluator=BC_evaluator,
        numFolds= 2)

    fitModel = crossval.fit(train)

    best_model = fitModel.bestModel
    predictions = fitModel.transform(test) #fitModel automatically uses best model
    area_under_roc = BC_evaluator.evaluate(predictions)
    print(f"Area under ROC: {area_under_roc}")

    client.log_metric(run.info.run_id, "Area under ROC", area_under_roc)
    logParameters(best_model)
    mlflow.spark.save_model(best_model, "model_"+str(Mtype))

# Terminate the client
client.set_terminated(run.info.run_id)

LogisticRegression
Area under ROC: 0.8818342151675485
GBTClassifier
